#Importing the required libraries

In [1]:
#using selenium library to automate the browser and test

from selenium import webdriver      #webdriver component provides a programing interface to work on 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup       #to parse the html document and scrape the necessary data

import time

#Connecting to the drivers

In [2]:
serv_obj = Service("C:\Drivers\chromedriver-win64\chromedriver")        #using Service classes to manage the local drivers
serv_obj.start()

#WebScraping the data

In [6]:
data=[]

driver = webdriver.Remote(serv_obj.service_url)     #Remote class is being used to control the browser by sending commands
driver.get("https://www.linkedin.com/search/results/people/?heroEntityKey=urn%3Ali%3Aorganization%3A1063&keywords=cisco&origin=SWITCH_SEARCH_VERTICAL&position=0&searchId=b10384d8-0887-42a4-bba6-1badbaa94cd8&sid=*ek")
        #passing the url through get function
finding = driver.find_element(By.ID,"username").send_keys("enter your email/phone no.", Keys.ENTER)
finding = driver.find_element(By.ID,"password").send_keys("password", Keys.ENTER)
        #here we are locating the input cells to sign in by ID from html and passing the credentials through send keys function
time.sleep(5)       #providing the pausing time explicitly for the browser to load (could be adjusted as per requirements)

def scrape(n):      #creating a function to webscrape - n represents the number of linkedin profiles to be scraped
    for x in range(n):          #creating a loop to get details of profiles one by one
        profiles = driver.find_elements(By.CLASS_NAME, "DAiEIoxFQQOTOxMUJFJCZAGcsrPESGmwPvw")   #locating the class of list of profiles in results container
        profiles[x].click()     #indexing the profiles list to click and get further details
        time.sleep(5)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        full_name = soup.find('h1', class_='text-heading-xlarge inline t-24 v-align-middle break-words').get_text()
        occupation = soup.find('div', class_='text-body-medium break-words').get_text().strip()
                #scraping the data from profiles using class and tag name from html

        data.append({'full_name': full_name,'occupation': occupation})      #appending the details
        
        driver.back()       #to drive back to webpage of search results and to select the next profile
        time.sleep(5)
    
scrape(2)       #calling the function with required number of profiles to be scraped 
                #the number could be replaced with 'len(profiles)' to scrape all the profiles in the webpage


#def scroll(y):         #creating a function to scroll and move to next pages in search results - y represents the number of pages to be scraped
    #for i in range(y):
        #profiles = driver.find_elements(By.CLASS_NAME, "DAiEIoxFQQOTOxMUJFJCZAGcsrPESGmwPvw")
        #scrape(len(profiles))
        #page = driver.find_element(By.CLASS_NAME,'ember-view   ').send_keys(Keys.CONTROL + Keys.END)       #to scroll down to bottom of the page
        #time.sleep(5)
        #page = driver.find_element(By.CSS_SELECTOR,'button[aria-label = Next]')        #finding the 'Next' button in html using CSS_SELECTOR 
                                #providing tagname[unique attribute] to locate the element
        #page.click()           #takes to the next page
        #time.sleep(10)
#scroll(5)

import sqlite3          #importing sqlite3 module to save the scraped data in a database
connection = sqlite3.connect("Linkedin_profiles.db")        #connecting to database
cursor=connection.cursor()
sql_command=('''
        CREATE TABLE IF NOT EXISTS Profiles1 (
            full_name TEXT,
            occupation TEXT)''')
cursor.execute(sql_command)     #creating a table

for xyz in data:                #inserting the data using for loop
    cursor.execute('''INSERT INTO Profiles1 (full_name, occupation) VALUES (?, ?)''' ,
                    (xyz['full_name'], xyz['occupation']))


connection.commit()

cursor.execute('SELECT * FROM Profiles1')       
result = cursor.fetchall()
for row in result:
    print(row)          #to display the results
connection.close()


driver.quit()           #closing the browser

('Reena Thomas', 'Software Engineer III - Cisco')
('Lavanya Deivaa', 'Business Analyst @ Cisco | SQL | Python | Tableau | Snowflake | CEG ‘20')
